In [ ]:
import pandas as pd

In [ ]:
oneperson_households = pd.read_csv('data/one-person-households-edit.csv', delimiter=';')
trust_attitudes = pd.read_csv('data/self-reported-trust-attitudes-edit.csv', delimiter=';')
gdp_per_capita = pd.read_csv('data/gdp-per-capita-edit.csv', delimiter=';')
population = pd.read_csv('data/population-edit.csv', delimiter=';')
divorces = pd.read_csv('data/divorces-edit.csv', delimiter=';')
marriages = pd.read_csv('data/marriages-edit.csv', delimiter=';')

In [ ]:
oneperson_households

In [ ]:
gdp_per_capita_melt = (
    gdp_per_capita
        .rename(columns={'Country Name': 'Entity', 'Country Code': 'Code'})
        .drop(columns=['Indicator Name', 'Indicator Code', 'Unnamed: 68'])
        .melt(id_vars=['Entity', 'Code'], var_name='Year', value_name='GDP per Capita (current US$')
        .astype({'Year': 'int64'})
)

gdp_per_capita_melt.head()

In [ ]:
df_step1 = pd.merge(oneperson_households, gdp_per_capita_melt,
                       on=['Year', 'Code', 'Entity'], 
                       how='inner')

In [ ]:
population_melt = (
    population
        .rename(columns={'Country Name': 'Entity', 'Country Code': 'Code'})
        .drop(columns=['Indicator Name', 'Indicator Code', 'Unnamed: 68'])
        .melt(id_vars=['Entity', 'Code'], var_name='Year', value_name='Population')
        .astype({'Year': 'int64'})
)

In [ ]:
df_step2 = pd.merge(df_step1, population_melt,
                       on=['Year', 'Code', 'Entity'], 
                       how='inner')

df_step3 = pd.merge(df_step2, marriages,
                       on=['Year', 'Code', 'Entity'], 
                       how='left')

df_complete = pd.merge(df_step3, divorces,
                       on=['Year', 'Code', 'Entity'], 
                       how='left')

country_with_2018 = df_complete[df_complete['Year'] == 2018]['Entity'].unique()

df_complete = (
    df_complete[df_complete['Entity'].isin(country_with_2018)].rename(
        columns={'Crude marriage rate (per 1,000 inhabitants)': 'Marriage rate (per 1,000)',
             'Crude divorce rate (per 1,000 inhabitants)': 'Divorce rate (per 1,000)',
             'Entity': 'Country'}
    )
    .copy()
)

df_complete = df_complete[df_complete["Year"] >= 2000].copy()

all_years = pd.DataFrame({'Year': range(2000, 2019)})
countries = df_complete['Country'].unique()
countries_df = pd.DataFrame({'Country': countries})
full_index = countries_df.merge(all_years, how='cross')

df_final = pd.merge(full_index, df_complete, on=['Country', 'Year'], how='left')

df_final['Share of one person households'] = df_final['Share of one person households'].fillna(0).copy()

df_final

In [ ]:
df_final.to_csv('data/df_complete.csv', index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=74a8de96-ace1-4d06-94a9-e911afda420a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>